In [ ]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# load data
data = pd.read_csv("/content/drive/MyDrive/train_preprocess.tsv.txt", sep='\t', names=['text','label'])
data

In [ ]:
data.label.value_counts()

In [ ]:
# cek data
data.duplicated().sum()

In [ ]:
data = data.drop_duplicates()

In [ ]:
data = data.reset_index(drop=True)
data

In [ ]:
data.isna()

In [ ]:
data.isna().sum()

In [ ]:
# kolom baru total_char
len(data.text)

In [ ]:
data['total_char'] = data.text.apply(len)
data.head()

In [ ]:
# kolom baru total_word
print(data.text)

In [ ]:
data["total_word"] = data.text.apply(lambda words: len(words.split()))
data

In [ ]:
# descriptive statistic
data.corr()

In [ ]:
data.plot(x = "total_word",y = "total_char",kind = "scatter")

In [ ]:
data.groupby("label").total_word.sum().plot(kind='bar')

In [ ]:
data.groupby("label").total_char.sum().plot(kind='bar')

In [ ]:
# neural network
import re

def cleansing(txt):
  string = txt.lower()
  string = re.sub(r'[^a-zA-Z0-9]', ' ', string)
  return string

data["txt_clean"] = data.text.apply(cleansing)

In [ ]:
data.pop("text")
data.pop("total_char")
data.drop("total_word", axis=1, inplace=True)
data.head()

In [ ]:
data_preprocessed = data.txt_clean.tolist()
data_preprocessed

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
count_vect.fit(data_preprocessed)

X = count_vect.transform(data_preprocessed)
print ("Feature Extraction selesai")

In [ ]:
import pickle

pickle.dump(count_vect, open("feature.pkl", "wb"))

In [ ]:
from sklearn.model_selection import train_test_split

classes = data.label
X_train, X_test, y_train, y_test = train_test_split(X, classes, test_size = 0.2)

In [ ]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier() 
model.fit(X_train, y_train)

print ("Training selesai")

In [ ]:
from sklearn.metrics import classification_report

test = model.predict(X_test)

print ("Testing selesai")

print(classification_report(y_test, test)) 

In [ ]:
pickle.dump(model, open("model_neural.pkl", "wb"))

In [ ]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

kf = KFold(n_splits=5,random_state=42,shuffle=True)

accuracies = []

y = classes

for iteration, data in enumerate(kf.split(X), start=1):

    data_train   = X[data[0]]
    target_train = y[data[0]]

    data_test    = X[data[1]]
    target_test  = y[data[1]]

    clf = MLPClassifier()
    clf.fit(data_train,target_train)

    preds = clf.predict(data_test)
  
    accuracy = accuracy_score(target_test,preds)

    print("Training ke-", iteration)
    print(classification_report(target_test,preds))
    print("======================================================")

    accuracies.append(accuracy)

average_accuracy = np.mean(accuracies)

print()
print()
print()
print("Rata-rata Accuracy: ", average_accuracy)

In [ ]:
original_text =  '''
semua patut diapresiasi.
'''

loaded_transformer = pickle.load(open('/content/feature.pkl', 'rb'))
text = loaded_transformer.transform([cleansing(original_text)])

loaded_model = pickle.load(open('/content/model_neural.pkl', 'rb'))
result = loaded_model.predict(text)[0]
print("Sentiment:")
print()
print(result)